# Volt Analytics

## Import Dependencies

In [1]:
import os
import datetime as dt
import numpy as np
import pandas as pd
import openpyxl as Workbook

pd.set_option('display.max_rows', 500)

In [2]:
currentDate =  dt.datetime.today().date()
currentDate

datetime.date(2017, 9, 19)

## Read in CSV files

In [3]:
#Volt Daily Data
voltDailyData = pd.read_csv('data/voltdailydata_' + currentDate.strftime('%Y%m%d') + '.csv')
voltDailyData['Date'] = pd.to_datetime(voltDailyData['Date'])
voltDailyData['Date'] = voltDailyData['Date'].dt.date

In [4]:
voltDailyData.head()

,Date,MilesDriven,EvMilesDriven,GallonsBurned
0,2017-06-25,163.7,52.6,3.03005
1,2017-06-26,202.4,42.5,4.15543
2,2017-06-27,202.3,53.8,3.97314
3,2017-06-28,197.7,52.4,3.93617
4,2017-06-29,192.6,45.2,3.83577


In [5]:
# Calculate total ICE (Internal Combustion Engine) miles driven
voltDailyData = voltDailyData.rename(columns = {'MilesDriven':'TotalMilesDriven'})
voltDailyData['TotalICEMiles'] = voltDailyData['TotalMilesDriven'] - voltDailyData['EvMilesDriven']
voltDailyData.head()

,Date,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles
0,2017-06-25,163.7,52.6,3.03005,111.1
1,2017-06-26,202.4,42.5,4.15543,159.9
2,2017-06-27,202.3,53.8,3.97314,148.5
3,2017-06-28,197.7,52.4,3.93617,145.3
4,2017-06-29,192.6,45.2,3.83577,147.4


In [6]:
# Create new DailyMPG Column, round values after adding new column
voltDailyData['DailyMPG'] = voltDailyData['TotalMilesDriven'] / voltDailyData['GallonsBurned']
voltDailyData['DailyMPG'] = pd.to_numeric(voltDailyData['DailyMPG'], errors='coerse')
voltDailyData = voltDailyData.replace(np.nan, 0)
voltDailyData = voltDailyData.round(2)
voltDailyData.head()

,Date,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles,DailyMPG
0,2017-06-25,163.7,52.6,3.03,111.1,54.03
1,2017-06-26,202.4,42.5,4.16,159.9,48.71
2,2017-06-27,202.3,53.8,3.97,148.5,50.92
3,2017-06-28,197.7,52.4,3.94,145.3,50.23
4,2017-06-29,192.6,45.2,3.84,147.4,50.21


In [7]:
# Add '250+' miles to any DailyMPG value that has a value of infinity
# Why 250+? That's the "infinity mileage" that the volt displays in car

voltDailyData = voltDailyData.replace(np.inf, '250+')
voltDailyData.tail()

,Date,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles,DailyMPG
62,2017-09-11,0.0,0.0,0.00,0.0,0
63,2017-09-12,196.6,0.0,4.93,196.6,39.86
64,2017-09-13,122.0,0.0,3.30,122.0,36.95
65,2017-09-18,36.2,36.2,0.00,0.0,250+
66,2017-09-19,31.8,31.8,0.00,0.0,250+


In [8]:
voltDailyData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 6 columns):
Date                67 non-null object
TotalMilesDriven    67 non-null float64
EvMilesDriven       67 non-null float64
GallonsBurned       67 non-null float64
TotalICEMiles       67 non-null float64
DailyMPG            67 non-null object
dtypes: float64(4), object(2)
memory usage: 3.2+ KB


## Volt Readings

In [9]:
#Volt Readings throughout the day (usually two), convert Timestamp to datetime object
voltReadings = pd.read_csv('data/voltdata_' + currentDate.strftime('%Y%m%d') + '.csv')
voltReadings.rename(columns = {'LocalTimestamp':'Date', 'GallonsBurned': 'LifeTimeGalBurned'}, inplace=True)
voltReadings['Date'] = pd.to_datetime(voltReadings['Date'])

voltReadings.head()

,BatteryLevel,ChargeState,OilLife,TirePressure_LF,TirePressure_LR,TirePressure_RF,TirePressure_RR,Date,LifeTimeGalBurned,LifetimeFuelEcon,LifetimeEvMiles,LifetimeMiles,LastTripFuelEcon,LastTripEvMiles,LastTripMiles,GallonsOfGas
0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,2017-09-19 18:26:49,760.2289,64.89379,20635.22148,49334.13224,0.0,0.0,0.0,3.17535
1,0.0,NaN,0.0,0.0,0.0,0.0,0.0,2017-09-18 19:30:13,760.2289,64.85196,20603.41349,49302.33046,0.0,0.0,0.0,3.17535
2,0.0,NaN,0.0,0.0,0.0,0.0,0.0,2017-09-18 06:39:52,760.2289,64.80439,20567.26211,49266.17287,0.0,0.0,0.0,3.17535
3,0.0,NaN,0.0,0.0,0.0,0.0,0.0,2017-09-17 18:44:08,760.2289,64.80438,20567.26211,49266.16044,0.0,0.0,0.0,3.21233
4,0.0,NaN,0.0,0.0,0.0,0.0,0.0,2017-09-17 06:02:14,760.2289,64.80438,20567.26211,49266.16044,0.0,0.0,0.0,3.21233


In [10]:
# Select only columns desired
voltReadings = voltReadings[['Date', 'LifeTimeGalBurned', 'LifetimeFuelEcon', 'LifetimeEvMiles','LifetimeMiles']]

In [11]:
#Calculate LifetimeICEMiles
voltReadings['LifetimeICEMiles'] = voltReadings['LifetimeMiles'] - voltReadings['LifetimeEvMiles']

In [12]:
# Sort based on timestamp, reorder columns, round values
voltReadings.sort_values(by='Date', inplace=True)
voltReadings = voltReadings[['Date', 'LifeTimeGalBurned','LifetimeFuelEcon','LifetimeEvMiles','LifetimeICEMiles','LifetimeMiles']]
voltReadings = voltReadings.round(2)
voltReadings

,Date,LifeTimeGalBurned,LifetimeFuelEcon,LifetimeEvMiles,LifetimeMiles,LifetimeICEMiles
167,2017-06-24 17:31:19,593.83,68.35,17941.91,40585.45,22643.54
166,2017-06-25 05:35:50,593.83,68.35,17941.91,40585.45,22643.54
165,2017-06-25 17:51:57,596.65,68.27,17988.50,40734.80,22746.30
164,2017-06-26 05:52:46,596.86,68.27,17994.46,40749.16,22754.70
163,2017-06-26 19:15:25,601.02,68.14,18036.93,40951.55,22914.62
162,2017-06-27 06:58:20,601.50,68.13,18049.11,40980.32,22931.22
161,2017-06-27 17:43:14,604.19,68.05,18084.05,41116.79,23032.74
160,2017-06-28 05:34:27,604.99,68.02,18090.77,41153.86,23063.09
159,2017-06-28 17:35:10,608.17,67.93,18130.94,41311.50,23180.56
158,2017-06-29 05:36:46,608.92,67.91,18143.16,41351.56,23208.41


In [13]:
voltReadings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168 entries, 167 to 0
Data columns (total 6 columns):
Date                 168 non-null datetime64[ns]
LifeTimeGalBurned    168 non-null float64
LifetimeFuelEcon     168 non-null float64
LifetimeEvMiles      168 non-null float64
LifetimeMiles        168 non-null float64
LifetimeICEMiles     168 non-null float64
dtypes: datetime64[ns](1), float64(5)
memory usage: 9.2 KB
